# Technical Assistance - INDIGO Social Investment / Fund Manager Prototypes
The sample data used in this visualisation relate to four impact bonds in which BII is an investor: Cornwall Frequent Attenders, Street Impact Brighton, Pyramid Project and Skill Mill.

This visualization represents the variety and quantity of technical assistance (TA) provided by the investor (BII) on each contract. 


## INDIGO database API endpoint
Setup INDIGO database API endpoint and helper methods for getting individual items from the API. This can be used with the `project`, `fund`, `organisation` and `assessment_resource` endpoints. Also, add method to simplify extracting values from the nested data dictionary.

In [ ]:
import requests
import textwrap

import plotly.graph_objects as go


INDIGO_DATABASE_API = 'https://golab-indigo-data-store.herokuapp.com/app/api1'


def api_get_item(endpoint, public_id):
    """
    Get individual item details from the API

    E.g. 
    item = api_get_item('project', 'INDIGO-POJ-0158')
    """
    try:
        response = requests.get(f'{INDIGO_DATABASE_API}/{endpoint}/{public_id}')
        item = response.json()
        return item
    except Exception as e:
        print(f'\nFailed to retrieve {endpoint} "{public_id}".\nError: {e}')
        return False


def extract_value(data, keys, default):
    """
    Safe method to get value from nested dictionary with default value fallback
    """
    try:
        result = data
        for key in keys:
            if result == default:
                break
            result = result.get(key) or default
        return result

    except Exception as e:
        print(f'Error: {e}')
        return default

## Visualisation data
This is the main method for generating the plot data. It is called for each project.

In [ ]:
def visualisation_data_social_investment_prototype_4(project_data):
    data = {}
    for technical_assistance_detail in extract_value(project_data, ['technical_assistance_details'], []):
        category = extract_value(technical_assistance_detail, ['category', 'value'], '')
        days = extract_value(technical_assistance_detail, ['number_of_days', 'value'], '0')
        if category and int(days) > 0:
            data[category] = int(days)
    return {'data': data}

## Plot helper method
Setup the helper method for adding defined colours to the bar plot.

In [ ]:
DEFAULT_BAR_CHART_COLOURS = [
    '#2f4b7c',
    '#00629b',
    '#3c77ab',
    '#665191',
    '#a05195',
    '#d45087',
    '#f95d6a',
    '#ff7c43',
    '#ffa600',
]

def get_bar_colour(index):
    return DEFAULT_BAR_CHART_COLOURS[index % len(DEFAULT_BAR_CHART_COLOURS)]

## Get project data
Now we actually call the INDIGO API 'project' endpoint and retrieve the data for each of the projects of interest.

In [ ]:
# Call the API and pull down the data for each project
# and store in a dictionary for use later.
public_ids = ['INDIGO-POJ-0158', 'INDIGO-POJ-0167', 'INDIGO-POJ-0194', 'INDIGO-POJ-0195']

data = {}
endpoint = 'project'

for public_id in public_ids:
    data[public_id] = api_get_item(endpoint, public_id)

## Generate plot data
Pass each project to the `visualisation_data_social_investment_prototype_4` method and build a plot with the return values if they are valid.

In [ ]:
# Calculate the prototype data for each
plot_data = []

 # Get Raw data
selected_ids = []
categories = {}

for public_id in public_ids:
    project_data = data[public_id]['project']['data']
    prototype = visualisation_data_social_investment_prototype_4(project_data)

    # Print out the return prototype data structure. It is possible that not
    # all projects have valid data available to plot.
    print(public_id, prototype)

    if not prototype or not prototype.get('data'):
       continue

    selected_ids.append(public_id)

    for category, days in prototype.get('data').items():
        if category not in categories:
            categories[category] = {}
        categories[category][public_id] = days

# Add the bar plots
if selected_ids:

    # Sort the categories by the first project with biggest values first
    sorted_category_titles = sorted(categories, key=lambda x: categories[x][selected_ids[0]], reverse=True)

    for index, category in enumerate(sorted_category_titles):
        category_data = categories[category]

        y_data = []

        for public_id in selected_ids:
            y_data.append(category_data.get(public_id, 0))

        plot_data.append(
            go.Bar(
                name='<br>'.join(textwrap.wrap(category, 40)),
                x=selected_ids,
                y=y_data,
                marker=go.bar.Marker(
                    color=get_bar_colour(index),
                    line={
                        'color': 'white',
                        'width': 2,
                    },
                ),
            )
        )

else:
    print('Technical assistance data is not available')
    

## Build the figure

In [ ]:
fig = go.Figure(data=plot_data)

fig.update_traces(
    hovertemplate='%{y} days',
)

fig.update_layout(
    barmode='stack',
    height=800,
    title='Technical assistance',
    title_x=0.5,
    xaxis_title="Projects",
    yaxis_title="Days",
    showlegend=True,
)

fig.show()

This visualization represents the variety and quantity of technical assistance (TA) provided by the investor (BII) on each contract. This is how BII has chosen to depict their provision of TA, and demonstrates the TA provided by BII for one specific project. This does not relate to all social investors in terms of levels of provision of TA, or ways of describing TA. In this version, this is showing TA provided at the pre-contract phase only. 

Each project will be a vertical bar on the x-axis. (In the current version, there is only one project, so only one bar.) The y-axis shows the number of days that the social investor spent providing TA to either the provider or commissioner. The data come from the "Technical Assistance Details" tab.  

In this version of the visualization, we show the different types of technical assistance provided by the investor for the Pyramid Project (INDIGO-POJ-0194). Different colors are used to represent the different types of TA provided (from the same list of TA types in `investment-tech-assist-timelines`). 

This shows that the majority (19 days) of the technical assistance was spent on "Revenue Strategy – Business Planning". Some examples of these types of TA are: practical support with fundraising, developing the impact strategy and theory of change, and business model development. The full list of TA categories developed by BII and the GO Lab is in the INDIGO Data Dictionary.

## Important Notice and Disclaimer on INDIGO Data
<sub><sup>
INDIGO data are shared for research and policy analysis purposes. INDIGO data can be used to support a range of insights, for example, to understand the social outcomes that projects aim to improve, the network of organisations across projects, trends, scales, timelines and summary information. The collaborative system by which we collect, process, and share data is designed to advance data-sharing norms, harmonise data definitions and improve data use. These data are NOT shared for auditing, investment, or legal purposes. Please independently verify any data that you might use in decision making. We provide no guarantees or assurances as to the quality of these data. Data may be inaccurate, incomplete, inconsistent, and/or not current for various reasons: INDIGO is a collaborative and iterative initiative that mostly relies on projects all over the world volunteering to share their data. We have a system for processing information and try to attribute data to named sources, but we do not audit, cross-check, or verify all information provided to us. It takes time and resources to share data, which may not have been included in a project’s budget. Many of the projects are ongoing and timely updates may not be available. Different people may have different interpretations of data items and definitions. Even when data are high quality, interpretation or generalisation to different contexts may not be possible and/or requires additional information and/or expertise. Help us improve our data quality: email us at indigo@bsg.ox.ac.uk if you have data on new projects, changes or performance updates on current projects, clarifications or corrections on our data, and/or confidentiality or sensitivity notices. Please also give input via the INDIGO Data Definitions Improvement Tool and INDIGO Feedback Questionnaire.
</sup></sub>
